###### Multi-region analysis 

In this notebook we analyse the ability of a model trained on a region A to infer the subgrid forcing to achieve the same task on a different region, say region B. 

In [ ]:
import mlflow
import xarray as xr
import matplotlib.pyplot as plt
import os,sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../../src/gz21_ocean_momentum'))
from utils import select_run

In [ ]:
plt.rcParams["figure.figsize"] = (15, 10)

In [ ]:
def plot_dataset(dataset : xr.Dataset, plot_type = None, *args, **kargs):
    """Calls the plot function of each variable in the dataset"""
    plt.figure(figsize = (20, 5 * int(len(dataset) / 2)))
    kargs_ = [dict() for i in range(len(dataset))]
    def process_list_of_args(name: str):
        if name in kargs:
            if isinstance(kargs[name], list):
                for i, arg_value in enumerate(kargs[name]):
                    kargs_[i][name] = arg_value
            else:
                for i in range(len(dataset)):
                    kargs_[i][name] = kargs[name]
            kargs.pop(name)
    process_list_of_args('vmin')
    process_list_of_args('vmax')
    for i, variable in enumerate(dataset):
        plt.subplot(int(len(dataset) / 2), 2, i + 1)
        if plot_type is None:
            try:
                # By default we set the cmap to coolwarm
                kargs.setdefault('cmap', 'coolwarm')
                dataset[variable].plot(*args, **kargs_[i], **kargs)
            except AttributeError as e:
                kargs.pop('cmap', None)
                dataset[variable].plot(*args, **kargs)
        else:
            plt_func = getattr(dataset[variable].plot, plot_type)
            plt_func(*args, **kargs)
import matplotlib.animation as animation

def dataset_to_movie(dataset : xr.Dataset, interval : int = 50,
                    *args, **kargs):
    """Generates animations for all the variables in the dataset"""
    fig = plt.figure(figsize = (20, 5 * int(len(dataset) / 2)))
    axes = list()
    ims = list()
    for i, variable in enumerate(dataset.keys()):
        axes.append(fig.add_subplot(int(len(dataset) / 2), 2, i + 1))
    for i, t in enumerate(dataset['time']):
        im = list()
        for axis, variable in zip(axes, dataset.keys()):
            plt.sca(axis)
            img = dataset[variable].isel(time=i).plot(vmin=-2, vmax=2,
                                                      cmap='coolwarm')
            cb = img.colorbar
            cb.remove()
            im.append(img)
        ims.append(im)
    ani = animation.ArtistAnimation(fig, ims, 
                                    interval=interval, blit=True,
                                    repeat_delay=1000)
    return ani

In [ ]:
client = mlflow.tracking.MlflowClient()
client.list_experiments()

In [ ]:
import pandas as pd
def select_run(limit=1000, sort_by=None, cols=None, merge=None, *args, **kargs):
    """Allows to select a run from the tracking store interactively"""
    mlflow_runs = mlflow.search_runs(*args, **kargs)
    if cols is None:
        cols = list()
    cols = ['run_id', 'experiment_id' ] + cols
    mlflow_runs = mlflow_runs.iloc[:limit]
    # Remove possible duplicate columns
    new_cols = list()
    for e in cols:
        if e not in new_cols:
            new_cols.append(e)
    cols = new_cols
    print(len(mlflow_runs))
    if merge is not None:
        cols[cols.index('run_id')] = 'run_id_x'
        cols[cols.index('experiment_id')] = 'experiment_id_x'
        for name, key_left, key_right in merge:
            experiment = mlflow.get_experiment_by_name(name)
            df2 = mlflow.search_runs(experiment_ids=experiment.experiment_id)
            mlflow_runs = pd.merge(mlflow_runs, df2, left_on=key_left,
                                   right_on=key_right)
    print(len(mlflow_runs))
    if len(mlflow_runs) == 0:
        raise Exception('No data found. Check that you correctly set \
                        the store')
    if sort_by is not None:
        mlflow_runs = mlflow_runs.sort_values(by=sort_by, ascending=False)
        cols.append(sort_by)
    print(mlflow_runs[cols])
    id_ = int(input('Run id?'))
    if id_ < 0:
        sys.exit()
    return mlflow_runs.loc[id_, :]

In [ ]:
cols = ['start_time_x','params.model_cls_name', 'metrics.test loss', 'params.lat_min', 
       'params.lat_max', 'params.long_min', 'params.long_max', 'params.n_epochs_x', 'params.model_run_id']
run = select_run(sort_by='start_time_x', cols=cols, merge=[('Unet', 'params.model_run_id', 'run_id'),
                                  ('forcingdata1pct', 'params.data_run_id', 'run_id')], experiment_ids = ['11',])

In [ ]:
for k,v in run.items():
    print(f'{k}: {v}')

In [ ]:
data_run_id = run['params.data_run_id']
data_run = client.get_run(data_run_id)

In [ ]:
from analysis.base import get_test_datasets
test_datasets = get_test_datasets(run['run_id_x'])

In [ ]:
id = 0
error = (test_datasets[id]['S_xpred'] - test_datasets[id]['S_x']) 
error0 = test_datasets[id]['S_x']

In [ ]:
((error**2).mean(dim='time') / (error0**2).mean(dim='time')).plot(vmin=0.2, vmax=1)

In [ ]:
import numpy as np
model_output = test_datasets[id]
model_output['S_xscale'] = 1/(model_output['S_xscale'])
model_output['S_yscale'] = 1/(model_output['S_yscale'])
model_output['err_S_x'] = (model_output['S_x'] - model_output['S_xpred'])**2
model_output['err_S_y'] = (model_output['S_y'] - model_output['S_ypred'])**2

In [ ]:
model_output['time_index'] = xr.DataArray(np.arange(len(model_output.coords['time'])),
                                       dims = ('time',),
                                       coords = {'time' : model_output['time']})
model_output = model_output.swap_dims({'time' : 'time_index'})

In [ ]:
from random import randint
n_times = len(model_output['time'])
random_time = randint(0, n_times)
print(random_time)
plot_dataset(model_output.isel(time_index=random_time)[['u_surf', 'v_surf', 'S_x', 'S_y', 'S_xpred', 'S_ypred',
                                                          'S_xscale', 'S_yscale', 'err_S_x', 'err_S_y']],
            vmin = [-2]*6 + [0., 0., 0., 0.], vmax = [2]*6 + [1, 1,1,1])

In [ ]:
(model_output['err_S_x']).mean(dim='time_index').plot(vmax=1)

In [ ]:
fig = plt.figure(figsize=(30, 30))
long = -45
lat = 44
plt.subplot(2, 1, 1)
time = slice(0, 600)
model_output['S_x'].isel(time_index=time).sel(longitude=long, latitude=lat, method='nearest').plot(linewidth=3)
model_output['S_xpred'].isel(time_index=time).sel(longitude=long, latitude=lat, method='nearest').plot(linewidth=3)
uB = model_output['S_xpred'] + 1.96 * model_output['S_xscale']
lB = model_output['S_xpred'] - 1.96 * model_output['S_xscale']
uB.isel(time_index=time).sel(longitude=long, latitude=lat, method='nearest').plot(linestyle='--',color='gray')
lB.isel(time_index=time).sel(longitude=long, latitude=lat, method='nearest').plot(linestyle='--',color='gray')
plt.legend(('Sx', 'Sx_pred'))

In [ ]:
correlations = (model_output['S_y'] * model_output['S_ypred']).mean(dim='time_index') / np.sqrt((model_output['S_y']**2).mean(dim='time_index') * (model_output['S_ypred']**2).mean(dim='time_index'))

In [ ]:
correlations.plot(vmin=0.1, vmax=1)